In [1]:
import os
import math
import shutil
import numpy as np
from keras.optimizers import RMSprop, SGD
from keras.losses import categorical_crossentropy
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import *
from keras.layers import *
import pandas as pd

Using TensorFlow backend.


In [2]:
%run '../lib.ipynb'
%run '../data-generator.ipynb'

In [3]:
# make only specific GPU to be utilized
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
HOLDOUT_PREDS = 'out/holdout/holdout_predictions_%d.npy'
HOLDOUT_Y = 'out/holdout/holdout_Y.npy'
FOLDS = 10

In [5]:
# read holdout predictions
h_preds = np.zeros((FOLDS, ) + np.load(HOLDOUT_PREDS % 0).shape)

for fold in range(FOLDS):
    h_preds[fold] = np.load(HOLDOUT_PREDS % fold)

h_preds = np.swapaxes(h_preds, 0, 1)
print('h_preds.shape:', h_preds.shape) # i, fold, label_1h

# read holdout Ys
holdout_Y = np.load(HOLDOUT_Y)

h_preds.shape: (4000, 10, 12)


In [7]:
# create model

# i = Input(shape=(h_preds.shape[1:]))
# x = Flatten()(i)
# x = Dense(512, activation='sigmoid')(x)
# x = Dropout(0.5)(x)
# x = Dense(256, activation='sigmoid')(x)
# x = Dropout(0.5)(x)
# x = Dense(128, activation='sigmoid')(x)
# x = Dropout(0.5)(x)
# x = Dense(64, activation='sigmoid')(x)
# x = Dropout(0.5)(x)
# x = Dense(h_preds.shape[-1], activation='softmax')(x)

i = Input(shape=(h_preds.shape[1:]))
x = Flatten()(i)
x = Dense(64, activation='sigmoid')(x)
x = Dense(32, activation='sigmoid')(x)
x = Dense(h_preds.shape[-1], activation='softmax')(x)

model = Model(inputs=[i], outputs=[x])
model.compile(
    optimizer=RMSprop(lr=1e-3),
    loss=categorical_crossentropy,
    metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 12)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                7744      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 12)                396       
Total params: 10,220
Trainable params: 10,220
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(
    x=h_preds,
    y=holdout_Y,
    epochs=1000,
    callbacks=[
        ReduceLROnPlateau(
            monitor='loss', factor=0.2, patience=3, min_lr=1e-11, verbose=1),
        EarlyStopping(
            monitor='loss',
            min_delta=0.0000001,
            patience=20,
            verbose=1,
            mode='auto')
    ])

Epoch 1/1000
4000/4000 [==============================] - 0s - loss: 2.1600 - acc: 0.6222     
Epoch 2/1000
4000/4000 [==============================] - 0s - loss: 1.2145 - acc: 0.9615     
Epoch 3/1000
4000/4000 [==============================] - 0s - loss: 0.5421 - acc: 0.9617     
Epoch 4/1000
4000/4000 [==============================] - 0s - loss: 0.2955 - acc: 0.9622     
Epoch 5/1000
4000/4000 [==============================] - 0s - loss: 0.2235 - acc: 0.9615     
Epoch 6/1000
4000/4000 [==============================] - 0s - loss: 0.1995 - acc: 0.9615     
Epoch 7/1000
4000/4000 [==============================] - 0s - loss: 0.1888 - acc: 0.9620     
Epoch 8/1000
4000/4000 [==============================] - 0s - loss: 0.1821 - acc: 0.9620     
Epoch 9/1000
4000/4000 [==============================] - 0s - loss: 0.1780 - acc: 0.9617     
Epoch 10/1000
4000/4000 [==============================] - 0s - loss: 0.1746 - acc: 0.9625     
Epoch 11/1000
4000/4000 [========================

4000/4000 [==============================] - 0s - loss: 0.0920 - acc: 0.9792     
Epoch 87/1000
4000/4000 [==============================] - 0s - loss: 0.0918 - acc: 0.9800     
Epoch 88/1000
4000/4000 [==============================] - 0s - loss: 0.0916 - acc: 0.9800     
Epoch 89/1000
4000/4000 [==============================] - 0s - loss: 0.0909 - acc: 0.9802     
Epoch 90/1000
4000/4000 [==============================] - 0s - loss: 0.0910 - acc: 0.9802     
Epoch 91/1000
4000/4000 [==============================] - 0s - loss: 0.0907 - acc: 0.9802     
Epoch 92/1000
4000/4000 [==============================] - 0s - loss: 0.0898 - acc: 0.9802     
Epoch 93/1000
4000/4000 [==============================] - 0s - loss: 0.0896 - acc: 0.9800     
Epoch 94/1000
4000/4000 [==============================] - 0s - loss: 0.0879 - acc: 0.9813     
Epoch 95/1000
4000/4000 [==============================] - 0s - loss: 0.0881 - acc: 0.9815     
Epoch 96/1000
4000/4000 [=============================

4000/4000 [==============================] - 0s - loss: 0.0715 - acc: 0.9840     
Epoch 162/1000
4000/4000 [==============================] - 0s - loss: 0.0715 - acc: 0.9840     
Epoch 163/1000
3840/4000 [===========================>..] - ETA: 0s - loss: 0.0694 - acc: 0.9841
Epoch 00162: reducing learning rate to 1.2800001059076749e-08.
4000/4000 [==============================] - 0s - loss: 0.0715 - acc: 0.9840     
Epoch 164/1000
4000/4000 [==============================] - 0s - loss: 0.0715 - acc: 0.9840     
Epoch 165/1000
4000/4000 [==============================] - 0s - loss: 0.0715 - acc: 0.9840     
Epoch 166/1000
3712/4000 [==========================>...] - ETA: 0s - loss: 0.0751 - acc: 0.9836
Epoch 00165: reducing learning rate to 2.5600002118153498e-09.
4000/4000 [==============================] - 0s - loss: 0.0715 - acc: 0.9840     
Epoch 167/1000
4000/4000 [==============================] - 0s - loss: 0.0715 - acc: 0.9840     
Epoch 168/1000
4000/4000 [====================

In [9]:
TEST_Y_FILE = 'out/test/test_predictions_%d.npy'
FNAMES_FILE = 'out/test/test_files.npy'
OUT_FILE = 'out/submission-v2-L2-B-2.csv'

In [10]:
# read predictions
preds_shape = np.load(TEST_Y_FILE % 0).shape
preds = np.zeros((FOLDS, ) + preds_shape)

for fold in range(FOLDS):
    preds[fold] = np.load(TEST_Y_FILE % fold)

preds = np.swapaxes(preds, 0, 1)

In [11]:
test_preds = model.predict(preds, verbose=1)

154880/158538 [============================>.] - ETA: 0s

In [12]:
fnames = np.load(FNAMES_FILE)

In [13]:
labels = [''] * len(fnames)

for i in range(len(labels)):
    labels[i]= LABELS[np.argmax(test_preds[i])]

In [14]:
df = pd.DataFrame.from_dict({'fname': fnames, 'label': labels})
df = df.set_index('fname')

In [15]:
df.to_csv(OUT_FILE, index='fname')